# Capacitor de Placas Concéntricas Elípticas

En el capacitor, la placa exterior se encuentra a un potencial eléctrico $V_{ext}(\nu)$, mientras que la placa interior se encuentra a un potencial $V_{int}(\nu)$. El problema de valores en la frontera descrito en coordenadas elípticas se define:

$$\frac{1}{\sinh^2(\mu)+\sin^2(\nu)} (\frac{\partial^2V}{\partial\mu^2} + \frac{\partial^2V}{\partial\nu^2}) = 0,    \forall(\mu,\nu) \in \Omega$$

$$V(0.1, \nu) = \sin(2\nu), \forall\nu\in[0, 2\pi]$$
$$V(1, \nu) = 0, \forall\nu\in[0, 2\pi]$$

donde $\Omega$ es el dominio acotado por $\mu \in [0.1, 1] \times \nu \in [0, 2\pi]$.

In [130]:
import numpy as np
import sympy as sp
from sympy import init_printing, Eq
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

init_printing()

In [57]:
Vij, Vi1j, Vij1, Vim1j, Vijm1, du, dv = sp.symbols(r'V_{i\,j} V_{i+1\,j} V_{i\,j+1} V_{i-1\,j} V_{i\,j-1} \triangle\mu \triangle\nu')
u, v = sp.symbols(r'\mu \nu')
Vij, Vi1j, Vij1, Vim1j, Vijm1, du, dv, u, v

(V_{i,j}, V_{i+1,j}, V_{i,j+1}, V_{i-1,j}, V_{i,j-1}, \triangle\mu, \triangle\nu, \mu, \nu)

### Escribamos el esquema de diferencias finitas 

In [80]:
laplacian = (((Vi1j - 2*Vij + Vim1j)/du**2) + ((Vij1 - 2*Vij + Vijm1)/dv**2))
sys = Eq(laplacian, 0)
sp.simplify(sys)

            2                                                   2                                        
\triangle\mu ⋅(V_{i,j+1} + V_{i,j-1} - 2⋅V_{i,j}) + \triangle\nu ⋅(V_{i+1,j} - 2⋅V_{i,j} + V_{i-1,j})    
───────────────────────────────────────────────────────────────────────────────────────────────────── = 0
                                                 2             2                                         
                                     \triangle\mu ⋅\triangle\nu                                          

Véase que el termino senosoidal nunca es 0, luego basta con verificar que el término diferencial es 0

Finalmente si nombramos $$\frac{(\triangle\mu)^2}{(\triangle\nu)^2} = \alpha$$

In [95]:
alpha = sp.Symbol(r'\alpha')
rhs = Vi1j - 2*Vij + Vim1j + (Vij1 - 2*Vij + Vijm1)*alpha
new_eq = sp.Eq(rhs, 0)
new_eq

El cual puede ser escrito como:

In [98]:
rhs = -2*(1+alpha)*Vij + Vi1j + Vim1j + alpha*Vij1 + alpha*Vijm1
final_eq = Eq(rhs, 0)
final_eq

Ahora con nuestro esquema de diferencias finitas listas y las condiciones de frontera armemos nuestro sistema global

## Solución del Potencial Eléctrico

In [184]:
def electric_potential(n_u1, n_u2, v_ext, v_int):
    # Divide the domain acording to n_u1 and n_u2
    
    mu = np.linspace(0.1, 1, n_u1)
    nu = np.linspace(0, 2*np.pi, n_u2)
    dmu = mu[1] - mu[0]
    dnu = nu[1] - nu[0]
    alpha = (dmu**2)/(dnu**2)
    tg = lambda i, j: i*(n_u2) + j # To global
    dim = n_u1 * n_u2
    
    A = csr_matrix((dim, dim))
    b = csr_matrix((dim, 1))
    
    ij = np.zeros((dim, 2))
    
    for k in range(dim):
        # First let's identify i, j in the global system
        i = k//n_u2
        j = k - (k//n_u2)*n_u2
        ij[k, 0] = i
        ij[k, 1] = j
        
        ### Condiciones de frontera
        if(i == 0): # Condición interna
            nu = dnu * j # + nu_inicial, pero nu inicial es 0
            A[k, k] = 1
            b[k, :] = v_int(nu)
        elif(i == n_u1 - 1): # Condición externa
            A[k, k] = 1
            b[k, :] = v_ext
        else: # Condición general, aplicar ecuación
            A[k, k] = -2*(1+alpha)
            A[k, tg(i+1, j)] = 1
            A[k, tg(i-1, j)] = 1
            A[k, tg(i, j+1)] = alpha
            A[k, tg(i, j-1)] = alpha
            
    
    potential = spsolve(A, b)
    potential = potential.reshape(-1, 1)
        
    return np.hstack((ij, potential))

In [186]:
n_u = 20
n_v = 10
v_ext = 3
v_int = lambda v: np.sin(2*v)

e_potential = electric_potential(n_u, n_v, v_ext, v_int)


#with np.printoptions(threshold=np.inf):
#print(ans)

## Solución Campo Eléctrico

De electromagnetismo conocemos que para calcular el campo eléctrico podemos uilizar la expresión: 
$$\vec{E} = -\nabla V$$

In [127]:
to_global = lambda i, j: i*(n_v) + j 

In [163]:
mu = np.linspace(0.1, 1, 50)
mu

array([0.1       , 0.11836735, 0.13673469, 0.15510204, 0.17346939,
       0.19183673, 0.21020408, 0.22857143, 0.24693878, 0.26530612,
       0.28367347, 0.30204082, 0.32040816, 0.33877551, 0.35714286,
       0.3755102 , 0.39387755, 0.4122449 , 0.43061224, 0.44897959,
       0.46734694, 0.48571429, 0.50408163, 0.52244898, 0.54081633,
       0.55918367, 0.57755102, 0.59591837, 0.61428571, 0.63265306,
       0.65102041, 0.66938776, 0.6877551 , 0.70612245, 0.7244898 ,
       0.74285714, 0.76122449, 0.77959184, 0.79795918, 0.81632653,
       0.83469388, 0.85306122, 0.87142857, 0.88979592, 0.90816327,
       0.92653061, 0.94489796, 0.96326531, 0.98163265, 1.        ])